# README
## Options:
### Pretrained model:
- word2vec-ruscorpora-300	
- glove-twitter-200

### Tokenization: How to manage OOV?
- Link: '&'
- Tag: '#'
- Mention: '@' 

In [229]:
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', 400)

## Collective Code

In [250]:
import copy
import pandas as pd
import numpy as np
import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords as stpdfa
from nltk.stem.porter import PorterStemmer

import gensim
import gensim.models.word2vec as w2v
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


In [ ]:
model = api.load('glove-twitter-200')

In [251]:
def vectorize(line):
    words = []
    for word in line:  # line - iterable, for example list of tokens
        try:
            w2v_idx = w2v_indices[word]
        except KeyError:  # if you does not have a vector for this word in your w2v model, continue
            words.append(list(np.zeros(200,)))
            continue
        words.append(list(w2v_vectors[w2v_idx]))
        if not word:
            words.append(None)

        if len(line) > len(words):
            continue
    return np.asarray(words)


def get_W2V_AVG(raw_data):
    tweet_tokenizer = TweetTokenizer()
    tweet_tokens = []

    for sent in raw_data.text:
        sent = re.sub(r"http\S+", "&", sent)
        sent = re.sub(r"@\S+", "@", sent)
        sent = re.sub(r"#\S+", "#", sent)
        sent = re.sub(r'([^\s\w@#&]|_)+', '', sent)
        sent = re.sub('', '', sent.lower())
        # print(tweet_tokenizer.tokenize(sent))
        sent = [tweet_tokenizer.tokenize(sent)]
        # sent = [tweet_tokenizer.tokenize(sent.lower())]
        tweet_tokens.append(sent)
        # tweet_tokens.append(tweet_tokenizer.tokenize(sent))
    df_tokens = pd.DataFrame(tweet_tokens, columns=['token'])
    df_tokens['token_vec'] = copy.deepcopy(df_tokens['token'])

    for index, sent in enumerate(df_tokens['token_vec']):
        df_tokens['token_vec'][index] = vectorize(sent).mean(axis=0)

    # df_test[['text_token','text_token_vec']].head()

    df_temp = pd.DataFrame(
        df_tokens['token_vec'].values.tolist()).add_prefix('vec_avg')

    df_tokens = df_tokens.join(df_temp).drop('token_vec', axis=1)
    # df_test.drop('text_token_vec',axis=1, inplace=True)
    return df_tokens

In [252]:
raw_data = pd.read_csv("./data/_PHEME_text.csv")
raw_RHI = pd.read_csv("./data/_RHS_text.csv")

In [253]:
PHEME_W2V_AVG = get_W2V_AVG(raw_data)

# Import data

In [219]:
raw_data = pd.read_csv("./data/_PHEME_text.csv")

In [9]:
print(raw_data.shape)
raw_data.sample(5)

(5802, 2)


,text,Event
1080,"#ICYMI, Christopher Hitchens on the case for m...",charliehebdo
4120,DEVELOPING NEWS: Soldier shot at War Memorial....,ottawashooting
5004,RT @tomsteinfort: Terrifying photo of hostages...,sydneysiege
2305,#Ferguson chief said the officer was unaware o...,ferguson
4898,BREAKING: 2 people have run out of Sydney buil...,sydneysiege


# Preprocessing

# Tokenization

In [243]:
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()
tweet_tokens = []

for sent in raw_data.text:
    sent = re.sub(r"http\S+", "&", sent)
    sent = re.sub(r"@\S+", "@", sent)
    sent = re.sub(r"#\S+", "#", sent)
    sent = re.sub(r'([^\s\w@#&]|_)+','', sent)
    sent = re.sub('','', sent.lower())
    # print(tweet_tokenizer.tokenize(sent))
    sent = [tweet_tokenizer.tokenize(sent)]
    # sent = [tweet_tokenizer.tokenize(sent.lower())]
    tweet_tokens.append(sent)
    # tweet_tokens.append(tweet_tokenizer.tokenize(sent))
df_tokens = pd.DataFrame(tweet_tokens, columns=['token'])


In [237]:
df_tokens.head()

,token
0,"[breaking, armed, man, takes, hostage, in, kosher, grocery, east, of, paris, &]"
1,"[#, killers, dead, confirmed, by, gendarmerie]"
2,"[top, french, cartoonists, charb, cabu, wolinski, tignous, confirmed, among, dead, in, #, #, attack, editor, is, critically, wounded]"
3,"[police, have, surrounded, the, area, where, the, #, attack, suspects, are, believed, to, be, &, &]"
4,"[photo, armed, gunmen, face, police, officers, near, #, hq, in, paris, &, &]"


# Word2Vec

## Downloading Pretrained model for Gensim

## Fetching pretrained Model and Convert the Raw Text

In [230]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
# corpus = api.load('text8')
# wv = api.load('word2vec-google-news-300')
# fasttext-wiki-news-subwords-300'
#  'glove-twitter-200',
model = api.load('glove-twitter-200')

In [20]:
# import inspect
# print(inspect.getsource(wv.__class__))

In [42]:
w2v_object = model.wv
w2v_vectors = w2v_object.vectors # here you load vectors for each word in your model
w2v_indices = {word: w2v_object.vocab[word].index for word in w2v_object.vocab} # here you load indices - with whom you can find an index of the particular word in your model 

In [241]:
def vectorize(line): 
    words = []
    for word in line: # line - iterable, for example list of tokens 
        try:
            w2v_idx = w2v_indices[word]
        except KeyError: # if you does not have a vector for this word in your w2v model, continue 
            words.append(list(np.zeros(200,)))
            continue
        words.append(list(w2v_vectors[w2v_idx]))
        if not word:
            words.append(None)

        if len(line) > len(words):
            continue
    return np.asarray(words)

In [233]:
# print("Tweet 1: ", raw_data['text'][1])
# print("Indice of '{}': {}".format(df_tokens['token'][1][0], w2v_indices[df_tokens['token'][1][0]]))
# # print("Indice of '{}': {}".format(raw_data['text_token'][1][0], w2v_vectors[w2v_indices[raw_data['text_token'][1][0]]]))
# # print("Indice of '{}': {}".format(raw_data['text_token'][1][1], w2v_indices[raw_data['text_token'][1][1]]))
# # print("Indice of '{}': {}".format(raw_data['text_token'][1][1], w2v_vectors[w2v_indices[raw_data['text_token'][1][1]]]))
# # print("\nVector of the first headline:\n", vectorize(raw_data['text_token'][1]))

In [244]:
import copy
df_tokens['token_vec'] = copy.deepcopy(df_tokens['token'])

for index, sent in enumerate(df_tokens['token_vec']):
    df_tokens['token_vec'][index] = vectorize(sent).mean(axis=0)

# df_test[['text_token','text_token_vec']].head()

df_temp = pd.DataFrame(df_tokens['token_vec'].values.tolist()).add_prefix('vec_avg')

df_tokens = df_tokens.join(df_temp).drop('token_vec',axis=1)
# df_test.drop('text_token_vec',axis=1, inplace=True)

In [245]:
df_tokens.head()

,token,vec_avg0,vec_avg1,vec_avg2,vec_avg3,vec_avg4,vec_avg5,vec_avg6,vec_avg7,vec_avg8,vec_avg9,vec_avg10,vec_avg11,vec_avg12,vec_avg13,vec_avg14,vec_avg15,vec_avg16,vec_avg17,vec_avg18,vec_avg19,vec_avg20,vec_avg21,vec_avg22,vec_avg23,vec_avg24,vec_avg25,vec_avg26,vec_avg27,vec_avg28,vec_avg29,vec_avg30,vec_avg31,vec_avg32,vec_avg33,vec_avg34,vec_avg35,vec_avg36,vec_avg37,vec_avg38,vec_avg39,vec_avg40,vec_avg41,vec_avg42,vec_avg43,vec_avg44,vec_avg45,vec_avg46,vec_avg47,vec_avg48,vec_avg49,vec_avg50,vec_avg51,vec_avg52,vec_avg53,vec_avg54,vec_avg55,vec_avg56,vec_avg57,vec_avg58,vec_avg59,vec_avg60,vec_avg61,vec_avg62,vec_avg63,vec_avg64,vec_avg65,vec_avg66,vec_avg67,vec_avg68,vec_avg69,vec_avg70,vec_avg71,vec_avg72,vec_avg73,vec_avg74,vec_avg75,vec_avg76,vec_avg77,vec_avg78,vec_avg79,vec_avg80,vec_avg81,vec_avg82,vec_avg83,vec_avg84,vec_avg85,vec_avg86,vec_avg87,vec_avg88,vec_avg89,vec_avg90,vec_avg91,vec_avg92,vec_avg93,vec_avg94,vec_avg95,vec_avg96,vec_avg97,vec_avg98,vec_avg99,vec_avg100,vec_avg101,vec_avg102,vec_avg103,vec_avg104,vec_avg105,vec_avg106,vec_avg107,vec_avg108,vec_avg109,vec_avg110,vec_avg111,vec_avg112,vec_avg113,vec_avg114,vec_avg115,vec_avg116,vec_avg117,vec_avg118,vec_avg119,vec_avg120,vec_avg121,vec_avg122,vec_avg123,vec_avg124,vec_avg125,vec_avg126,vec_avg127,vec_avg128,vec_avg129,vec_avg130,vec_avg131,vec_avg132,vec_avg133,vec_avg134,vec_avg135,vec_avg136,vec_avg137,vec_avg138,vec_avg139,vec_avg140,vec_avg141,vec_avg142,vec_avg143,vec_avg144,vec_avg145,vec_avg146,vec_avg147,vec_avg148,vec_avg149,vec_avg150,vec_avg151,vec_avg152,vec_avg153,vec_avg154,vec_avg155,vec_avg156,vec_avg157,vec_avg158,vec_avg159,vec_avg160,vec_avg161,vec_avg162,vec_avg163,vec_avg164,vec_avg165,vec_avg166,vec_avg167,vec_avg168,vec_avg169,vec_avg170,vec_avg171,vec_avg172,vec_avg173,vec_avg174,vec_avg175,vec_avg176,vec_avg177,vec_avg178,vec_avg179,vec_avg180,vec_avg181,vec_avg182,vec_avg183,vec_avg184,vec_avg185,vec_avg186,vec_avg187,vec_avg188,vec_avg189,vec_avg190,vec_avg191,vec_avg192,vec_avg193,vec_avg194,vec_avg195,vec_avg196,vec_avg197,vec_avg198,vec_avg199
0,"[breaking, armed, man, takes, hostage, in, kosher, grocery, east, of, paris, &]",-0.200159,-0.022104,0.169316,0.162104,-0.079669,-0.067645,0.134522,-0.100513,0.025423,-0.129884,-0.088205,0.168558,-0.766855,0.259602,0.113695,-0.058642,0.080902,-0.153879,-0.066224,-0.192514,0.070593,-0.136879,-0.091076,-0.039435,0.044066,0.837574,0.195193,-0.062630,-0.134976,-0.005924,-0.172016,-0.142005,0.004670,-0.190249,0.158722,0.286581,-0.055702,0.136685,0.286701,-0.113899,0.078847,-0.054876,-0.010237,-0.012731,0.089428,0.088139,0.042980,-0.051393,-0.054021,-0.008542,-0.460845,-0.054243,-0.056661,0.030779,-0.312091,0.075869,-0.159379,-0.002325,-0.121169,0.090237,0.037765,0.096103,-0.017929,-0.071445,0.214892,0.046556,0.197175,0.061560,-0.024521,-0.056524,-0.190097,-0.023057,0.099621,0.064687,0.193431,0.111219,-0.071651,-0.083748,-0.017425,-0.042494,0.402205,-0.050672,0.189620,0.146298,-0.077849,-0.114381,-0.103333,0.153981,-0.006182,-0.100918,-0.001299,-0.060657,0.167663,0.015330,-0.077277,0.054065,0.090626,-0.056983,0.122849,0.071176,0.229443,-0.101146,-0.038592,-0.199690,0.206279,-0.309849,0.093074,0.038219,-0.019591,-0.223114,0.124421,-0.049038,0.065194,-0.059252,0.020795,-0.216016,-0.058488,0.106349,-0.200736,0.281002,0.012990,-0.030815,-0.038219,-0.002121,-0.177230,-0.099608,0.058361,0.116085,0.389631,0.341496,-0.210511,0.076919,-0.145283,-0.323431,0.018985,-0.262398,0.133821,0.083993,0.109703,-0.085828,0.019022,0.165334,-0.211419,-0.107355,-0.183889,0.018511,-0.025599,-0.037882,-0.033412,0.171158,0.078859,-0.005609,-3.652662,-0.108474,0.244953,-0.066086,0.015625,0.100949,0.034245,-0.002728,-0.086317,0.241056,-0.008944,0.000277,0.242234,-0.132559,-0.143875,0.003110,0.350015,-0.237144,-0.169752,0.072902,0.052614,0.212688,-0.025674,-0.455150,0.175094,-0.065420,-0.309130,0.095305,-0.082957,-0.135087,-0.187487,0.070619,0.023726,0.096496,0.136371,-0.261459,0.068401,0.110441,0.05372